In [6]:
import torch
from nilearn import connectome
from nilearn.datasets import fetch_abide_pcp
import pandas as pd

In [ ]:
EXEC = False        # Turn to True if you download the dataset for the first time

if EXEC : 
    abide_data_dir = './data/' + 'ABIDE_dataset'       
    num_subjects = 871       # Number of subjects for the ABIDE dataset


    abide_data = fetch_abide_pcp(data_dir = abide_data_dir,
                                n_subjects = num_subjects, 
                                pipeline = 'cpac',
                                derivatives = ['rois_ho'], 
                                quality_checked = True, 
                                verbose = 1,
                                band_pass_filtering = True)

In [10]:
phenotypic_data = pd.read_csv('data/ABIDE_dataset/ABIDE_pcp/Phenotypic_V1_0b_preprocessed1.csv')